## Preprocessing

In [30]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("Resources/charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


#### Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

In [31]:
app_df = app_df.drop(columns=["EIN", "NAME"])
app_df.head(1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1


In [32]:
app_df.shape

(34299, 10)

#### Determine the number of unique values in each column.

In [33]:
app_cat = app_df.dtypes.index.tolist()
app_df[app_cat].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

#### Look at the APPLICATION_TYPE value counts for binning

In [34]:
type_counts = app_df["APPLICATION_TYPE"].value_counts()
type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

#### Choose a cutoff value and create a list of application types to be replaced

In [35]:
Infinity = np.inf
app_type_bins = type_counts.value_counts(bins=[0, 500, Infinity])
app_type_bins

(-0.001, 500.0]    9
(500.0, inf]       8
Name: APPLICATION_TYPE, dtype: int64

In [36]:
app_df["COUNT"] = app_df.groupby(["APPLICATION_TYPE"])["APPLICATION_TYPE"].transform('count')
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,528
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,27037
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1173
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,27037
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,27037


In [37]:
other_df = app_df.loc[app_df["COUNT"] < 500]
saved_df = app_df.loc[app_df["COUNT"] >= 500]  
other_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
41,T2,Independent,C1000,ProductDev,Trust,1,25000-99999,N,1329410,0,16
66,T9,Independent,C1000,ProductDev,Trust,1,10M-50M,N,9955466,1,156
195,T9,Independent,C1000,ProductDev,Co-operative,1,10M-50M,N,25175802,1,156
196,T9,Independent,C1000,ProductDev,Trust,1,0,N,5000,1,156
327,T13,Independent,C1000,Preservation,Co-operative,1,0,N,5000,1,66
...,...,...,...,...,...,...,...,...,...,...,...
34204,T9,Independent,C1000,ProductDev,Co-operative,1,100000-499999,N,2152031,1,156
34206,T9,Independent,C2000,ProductDev,Co-operative,1,5M-10M,N,8588000,1,156
34210,T9,Independent,C1000,ProductDev,Trust,1,1M-5M,N,5779336,1,156
34216,T15,Independent,C1000,ProductDev,Trust,1,100000-499999,N,131790,1,2


In [38]:
saved_df.shape

(34023, 11)

In [39]:
other_df.shape

(276, 11)

In [40]:
keep_type = saved_df["APPLICATION_TYPE"].value_counts()
keep_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
Name: APPLICATION_TYPE, dtype: int64

#### Use the variable name `application_types_to_replace`

In [41]:
application_types_to_replace = other_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace

T9     156
T13     66
T12     27
T2      16
T25      3
T14      3
T29      2
T15      2
T17      1
Name: APPLICATION_TYPE, dtype: int64

In [42]:
other_df.loc[other_df["APPLICATION_TYPE"] != "Shirley", "APPLICATION_TYPE"] = "Other"
new_type = other_df["APPLICATION_TYPE"].value_counts()
new_type


Other    276
Name: APPLICATION_TYPE, dtype: int64

# <span style="color: blue;">CURRENTLY</span> <span style="color: green;">COPYING</span> <span style="color: red;">HERE</span>

In [43]:
other_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
41,Other,Independent,C1000,ProductDev,Trust,1,25000-99999,N,1329410,0,16
66,Other,Independent,C1000,ProductDev,Trust,1,10M-50M,N,9955466,1,156
195,Other,Independent,C1000,ProductDev,Co-operative,1,10M-50M,N,25175802,1,156
196,Other,Independent,C1000,ProductDev,Trust,1,0,N,5000,1,156
327,Other,Independent,C1000,Preservation,Co-operative,1,0,N,5000,1,66


In [44]:
application_df = pd.concat([saved_df, other_df], axis=0)
application_df = application_df.sort_index(ascending=True)
application_df = application_df.drop(columns=["COUNT"])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [45]:
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [46]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [47]:
# Choose a cutoff value and create a list of classifications to be replaced
class_type_bins = class_counts.value_counts(bins=[0, 1800, Infinity])
class_type_bins

(-0.001, 1800.0]    66
(1800.0, inf]        5
Name: CLASSIFICATION, dtype: int64

In [49]:
application_df["COUNT"] = application_df.groupby(["CLASSIFICATION"])["CLASSIFICATION"].transform('count')
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,528
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,27037
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1173
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,27037
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,27037


In [50]:
other_df = app_df.loc[application_df["COUNT"] < 1800]
saved_df = app_df.loc[application_df["COUNT"] >= 1800]  
other_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
13,T3,Independent,C2700,Preservation,Trust,1,25000-99999,N,5301,1,27037
22,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,318296,0,27037
44,T3,Independent,C7200,Preservation,Trust,1,0,N,5000,1,27037
48,T3,Independent,C1700,Preservation,Trust,1,0,N,5000,1,27037
69,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,6501,1,27037
...,...,...,...,...,...,...,...,...,...,...,...
34165,T3,CompanySponsored,C1700,Preservation,Association,1,25000-99999,N,698028,0,27037
34185,T3,Independent,C7000,Preservation,Trust,1,25000-99999,N,4035050,0,27037
34212,T3,Independent,C1700,Preservation,Trust,1,25000-99999,N,918970,1,27037
34219,T3,Independent,C7000,ProductDev,Trust,1,0,N,5000,0,27037


In [51]:
saved_df.shape

(32038, 11)

In [52]:
other_df.shape

(2261, 11)

In [55]:
other_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,COUNT
13,T3,Independent,C2700,Preservation,Trust,1,25000-99999,N,5301,1,27037
22,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,318296,0,27037
44,T3,Independent,C7200,Preservation,Trust,1,0,N,5000,1,27037
48,T3,Independent,C1700,Preservation,Trust,1,0,N,5000,1,27037
69,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,6501,1,27037


In [53]:
keep_class = saved_df["CLASSIFICATION"].value_counts()
keep_class

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

#### Use the variable name `classifications_to_replace`

In [56]:
classifications_to_replace = other_df["CLASSIFICATION"].value_counts()
classifications_to_replace

C7000    777
C1700    287
C4000    194
C5000    116
C1270    114
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: CLASSIFICATION, Length: 66, dtype: int64

In [57]:
other_df.loc[other_df["CLASSIFICATION"] != "Shirley", "CLASSIFICATION"] = "Other"
new_class = other_df["CLASSIFICATION"].value_counts()
new_class

Other    2261
Name: CLASSIFICATION, dtype: int64

In [58]:
application_df = pd.concat([saved_df, other_df], axis=0)
application_df = application_df.sort_index(ascending=True)
application_df = application_df.drop(columns=["COUNT"])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [59]:
# Check to make sure binning was successful
new_class_counts = application_df['CLASSIFICATION'].value_counts()
new_class_counts

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

# <span style="color: blue;">CURRENTLY</span> <span style="color: green;">WORKING</span> <span style="color: red;">HERE</span>

In [28]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE

In [31]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

In [32]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
#  YOUR CODE GOES HERE

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [36]:
# Train the model
#  YOUR CODE GOES HERE

Epoch 1/100
25724/25724 [==============================] - 2s 80us/sample - loss: 0.5700 - acc: 0.7234
Epoch 2/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5556 - acc: 0.7294
Epoch 3/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5529 - acc: 0.7304
Epoch 4/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5514 - acc: 0.7318
Epoch 5/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5495 - acc: 0.7322
Epoch 6/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5488 - acc: 0.7330
Epoch 7/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5482 - acc: 0.7335
Epoch 8/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5475 - acc: 0.7326
Epoch 9/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5468 - acc: 0.73491s - lo
Epoch 10/100
25724/25724 [==============================] - 1s 49u

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391


In [38]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE